# Calling libraries

In [1]:
import os
import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import albumentations as A
import glob
import segmentation_models as sm
from sklearn.metrics import classification_report
sm.set_framework('tf.keras')
sm.framework()

Segmentation Models: using `keras` framework.


'tf.keras'

In [2]:
import GPUtil
import psutil
import tensorflow as tf

os.environ["CUDA_VISIBLE_DEVICES"]="0"

GPUtil.showUtilization()
psutil.cpu_percent()

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  1% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |


2.1

# Read data addresses

In [3]:
DATA_DIR = './bench_data/' 

x_train_dir = os.path.join(DATA_DIR, 'train_rgb/')
y_train_dir = os.path.join(DATA_DIR, 'train_anno/')

x_valid_dir = os.path.join(DATA_DIR, 'test_rgb/')
y_valid_dir = os.path.join(DATA_DIR, 'test_anno/')


# Necessary functions and classes

In [4]:
class Dataset:

    
    CLASSES = ['non', 'reservoir','manmade']
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None, 
            augmentation=None, 
            preprocessing=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        self.class_values = [self.CLASSES.index(cls.lower()) for cls in classes]
        
        self.augmentation = augmentation
        self.preprocessing = preprocessing
    
    def __getitem__(self, i):
        
        image = cv2.imread(self.images_fps[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[i], 0)
        
        masks = [(mask == v) for v in self.class_values]
        mask = np.stack(masks, axis=-1).astype('float')
        
        if mask.shape[-1] != 1:
            background = 1 - mask.sum(axis=-1, keepdims=True)
            mask = np.concatenate((mask, background), axis=-1)
        
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
            
        return image, mask
        
    def __len__(self):
        return len(self.ids)
    
    
class Dataloder(tf.keras.utils.Sequence):

    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(dataset))

        self.on_epoch_end()

    def __getitem__(self, i):
        
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])
        
        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        
        return batch
    
    def __len__(self):
        
        return len(self.indexes) // self.batch_size
    
    def on_epoch_end(self):
        
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)  
            

            
            
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0, 1)

def get_training_augmentation():
    train_transform = [

        A.HorizontalFlip(p=0.7),
        A.VerticalFlip(p=0.7),


        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

def denormalize(x):
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

def get_validation_augmentation():
    
    test_transform = [
        A.PadIfNeeded(384, 384)
    ]
    return A.Compose(test_transform)

def get_preprocessing(preprocessing_fn):

    
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

# Model training

In [5]:
BACKBONE = 'vgg16'
BATCH_SIZE = 8
CLASSES = [ 'reservoir', 'manmade']
LR = 0.0001
EPOCHS = 150

preprocess_input = sm.get_preprocessing(BACKBONE)

n_classes = len(CLASSES) + 1 
activation = 'softmax'

#create model                                            
model = sm.Unet(BACKBONE, classes=n_classes, activation=activation,encoder_weights='imagenet')


from tensorflow import keras
optim = keras.optimizers.Adam(LR)

dice_loss = sm.losses.DiceLoss(class_weights=np.array([1,1,1]))
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.Precision(threshold=0.5),sm.metrics.Recall(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

model_name =  './benchmark_unet.h5'

In [6]:
train_dataset = Dataset(
    x_train_dir, 
    y_train_dir, 
    classes=CLASSES, 
    augmentation=get_training_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

valid_dataset = Dataset(
    x_valid_dir, 
    y_valid_dir, 
    classes=CLASSES, 
    augmentation=get_validation_augmentation(),
    preprocessing=get_preprocessing(preprocess_input),
)

train_dataloader = Dataloder(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = Dataloder(valid_dataset, batch_size=1, shuffle=False)

assert train_dataloader[0][0].shape == (BATCH_SIZE,384, 384,3) 
assert train_dataloader[0][1].shape == (BATCH_SIZE,384, 384, n_classes)


callbacks = [                         
    keras.callbacks.ModelCheckpoint(model_name, save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=True),
]

In [7]:
history = model.fit_generator(
    train_dataloader, 
    steps_per_epoch=len(train_dataloader), 
    epochs= EPOCHS, 
    callbacks=callbacks, 
    validation_data=valid_dataloader, 
    validation_steps=len(valid_dataloader),
)


model.load_weights(model_name)

direction = glob.glob(x_valid_dir+'*.png')

dataset,d = valid_dataset, 'val_rgb/'

preds,annos = [], []
for i in range(len(dataset)):
    
    image, gt_mask = dataset[i] 
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    newimg = pr_mask[0,:,:,:]
    name = direction[i].replace('test_rgb','test_anno')
    
    p = newimg[:,:,0]+newimg[:,:,1]*2+newimg[:,:,2]*0
    
    preds.append(p)
    annos.append(cv2.imread(name)[:,:,0])
    
print('test classification_report ')    
print(classification_report(np.array(annos).flatten(), np.array(preds).flatten(), labels=[0,1,2], digits = 4))


direction = glob.glob(x_train_dir+'*.png')

dataset,d = train_dataset, 'train_rgb/'
preds,annos = [], []
for i in range(len(dataset)):
    
    image, gt_mask = dataset[i] 
    image = np.expand_dims(image, axis=0)
    pr_mask = model.predict(image).round()
    newimg = pr_mask[0,:,:,:]
    name = direction[i].replace('train_rgb','train_anno')
    
    p = newimg[:,:,0]+newimg[:,:,1]*2+newimg[:,:,2]*0
    preds.append(p)
    annos.append(cv2.imread(name)[:,:,0])
print('train report')    
print(classification_report(np.array(annos).flatten(), np.array(preds).flatten(), labels=[0,1,2], digits = 4))

/home/nayereh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/150
1474/1474 [==============================] - 653s 436ms/step - loss: 0.4626 - precision: 0.6062 - recall: 0.7091 - f1-score: 0.5990 - val_loss: 0.6055 - val_precision: 0.6502 - val_recall: 0.9016 - val_f1-score: 0.6203 - lr: 1.0000e-04
Epoch 2/150
1474/1474 [==============================] - 636s 431ms/step - loss: 0.3795 - precision: 0.6507 - recall: 0.7500 - f1-score: 0.6522 - val_loss: 0.6122 - val_precision: 0.6705 - val_recall: 0.8888 - val_f1-score: 0.6270 - lr: 1.0000e-04
Epoch 3/150
1474/1474 [==============================] - 638s 433ms/step - loss: 0.3599 - precision: 0.6642 - recall: 0.7650 - f1-score: 0.6674 - val_loss: 0.5901 - val_precision: 0.6890 - val_recall: 0.9092 - val_f1-score: 0.6606 - lr: 1.0000e-04
Epoch 4/150
1474/1474 [==============================] - 645s 437ms/step - loss: 0.3440 - precision: 0.6763 - recall: 0.7755 - f1-score: 0.6801 - val_loss: 0.5850 - val_precision: 0.6867 - val_recall: 0.9135 - val_f1-score: 0.6616 - lr: 1.0000e-04
Epoch 5/